## ch 5_11. 데이터 전처리하기

크롤링을 통해 수집한 뉴스 기사 데이터에서 필요없는 부분을 먼저 제거하겠습니다. 텍스트 데이터를 전처리할 때는 정규 표현식을 활용하면 좋습니다.

## 불필요한 텍스트 제거
- 기사 시작 부분에 언론사나 기자 이름을 언급하는 부분
- 기사 마지막 부분에 사진 출처를 언급하는 부분

In [191]:
!pip install tqdm

### 전처리 하기 전 텍스트 확인

In [194]:
import csv
from tqdm import tqdm

data = []

with open("./data/baseball.csv", encoding="utf-8") as fr:
    reader = csv.reader(fr)
    next(reader)
    filtered_cnt = 0
    for row in tqdm(reader, total=100000):
        _, _, title, content = row
        
        if not title or not content:
            filtered_cnt += 1
            continue
        data.append(row)
print("total data:", len(data), "filtered_cnt", filtered_cnt)

102919it [00:01, 77601.91it/s]                                                                                     

total data: 102520 filtered_cnt 399


In [195]:
data[:3]

[['https://sports.news.naver.com/news?oid=477&aid=0000346920',
  '20220314',
  '"시프트를 좋아하지만, 금지해도 야구의 후퇴는 아니다"',
  '[스포티비뉴스=대구, 박성윤 기자] 메이저리그가 수비 시프트를 금지한다. 수비 시프트를 사랑하는 한화 이글스 카를로스 수베로 감독은 아쉬워하면서도 잃는 게 있으면 얻는 것도 있다고 설명했다.한화 수베로 감독은 13일 대구삼성라이온즈파크에서 삼성 라이온즈와 시범경기에 앞서 메이저리그 수비 시프트 금지에 대한 본인 생각을 이야기헀다. 메이저리그는 오는 2023년부터 수비 시프트를 금지한다. 새로운 CBA(노사협정) 개정으로 규칙이 바뀐 셈이다.KBO 리그에 변화는 없다. 그러나 메이저리그를 경험해본 수베로 감독은 시프트가 사라지는 것이 아쉽다. 그는 KBO 리그에서 가장 많은 시프트를 사용하는 감독 가운데 한 명이다. 타자 유형에 따라 공격적인 시프트를 보여준다. 2루수가 유격수 쪽에 치우치거나 우익수 쪽으로 나가는 것은 기본적인 시프트일 정도로 다양한 시프트를 활용해 타자의 안타 생산을 막는다.13일 수베로 감독은 "나는 수비 시프트를 좋아한다. 시프트는 야구의 일부다. 야구는 확률 싸움을 하는 스포츠다. 시프트는 확률에 따라 맞춰 움직여 아웃 카운트를 잡을 수 있다. 마음에 든다. 최근 생긴 방법도 아니다. 1980~1990년대부터 경기를 할 때 투수와 타자에 맞춰 수비 위치를 바꿨다. 투수의 투구와 타자의 타격 성향을 고려해 움직였다. 나는 시프트 자체를 높게 평가한다"며 야구를 잘하는 수단으로 시프트가 좋다고 목소리를 높였다.메이저리그에는 수베로 감독만큼 시프트를 즐겨 사용하는 감독들이 많다. 극단적으로 당겨치는 왼손 타자를 사대하기 위해 내야수 4명 전원을 1루와 2루 사이에 배치한 장면도 있다. 메이저리그 강타자 배리 본즈는 시프트가 있어도 신경 쓰지 않고 본인의 타격을 한 것으로 유명하다. 투수들은 간혹 정상 수비였다면 아웃되는 타구였는데, 시프트

## 필터링에 사용할 정규 표현식 작성 요령

1. 예시 데이터를 가지고 정규표현식을 작성한 뒤, 결과를 확인한다.
2. 여러 데이터 샘플들에 대해서도 정규표현식이 의도한 대로 동작하는지 확인한다.

In [ ]:
example_article = "[마이데일리 = 장윤호 기자]롯데 자이언츠의 거인 이대호(40)의 꿈이 무너져가고 있다. 이대호가 새해 40세 불혹(不惑)의 나이가 됐다. 불혹은 주위에서 어떤 일이 벌어져도 중심을 잃지 않고 자신만의 원칙을 지켜 나갈 수 있는 경지에 오르는 시기다. 이번 스토브리그에 롯데 구단에 걱정스러운 일이 벌어지고 있다. 그런데 이대호는 어떤 대외 활동도 하지 않고 침묵하며 조용히 개인 훈련에 집중하는 모습이다. 불혹이 돼서인가? 지난 2017년 1월24일이다. 벌써 5년의 시간이 흘렀다. 롯데의 프랜차이즈 스타 이대호가 일본프로야구 미국 메이저리그를 거쳐 고향팀 롯데로 돌아왔다. 그는 ‘조선의 4번타자’답게 단숨에 최고가 됐다. 삼성에서 FA가 된 최형우가 고향팀 KIA 타이거즈와 4년 계약을 하면서 기록한 총액 100억 원을 훨씬 넘어 150억 원에 롯데와 계약했다. 당시 인터뷰에서 이대호는 ‘메이저리그에서 열심히 노력해 꿈을 이루었다. 남은 것은 롯데로 돌아와 함께 우승을 하는 것이다. 마지막 소원이 롯데의 우승’이라고 밝혔다. 2001년 롯데에 2차 1순위로 입단해 2011시즌까지 11시즌 동안 이대호는 무려 225개의 홈런을 쏘아 올렸다. 그리고 2008~2011 시즌까지 4년 연속 롯데를 포스트시즌으로 이끌었으나 한국시리즈 우승을 못하고 일본 프로야구로 떠났다. 그 때도 롯데에 서운함이 있었으나 말 없이 덮었다. 4년의 계약 기간이 지나갔다. 여전히 롯데는 한국시리즈 근처에 가지 못했다. 이대호는 올해 1월29일 롯데와 2년 총액 26억 원(계약금 8억 원 연봉 8억 원)에 특별한 옵션을 걸었다. 남은 2억 원에 다른 인센티브 조건이 아니라 한국시리즈 우승 의지를 담았다. 그리고 ‘우승하면 받는 1억 원의 옵션을 불우이웃을 돕는데 모두 기부하겠다’고 약속했다. 이대호는 그 때 계약이 늦어져 팬들에게 죄송하다고 말했다. 롯데 팬들이 걱정한 것도 사실이다. 전격 은퇴를 선언할 가능성도 분명히 있었다. 같은 시기 FA 4년 계약을 했던 KIA 최형우는 이미 12월에 3년 총액 47억원(게약금 13억원, 연봉 9억원, 옵션 7억원)에 계약을 맺었다.  차이가 분명히 있다. 최형우와 나이는 1살차였지만 이대호는 4년 계약 기간 중 파워나 기여도에서 그에 뒤지지 않았다. 그런데 단 하나 최형우는 계약 첫 해인 2017시즌 김기태 감독을 도와 팀을 한국시리즈 우승으로 이끌었다. 2022시즌 이대호는 롯데와 17년을 동행하게 된다. 부산에서 태어나고 자란 이대호가 무려 17년이나 한국시리즈 우승을 못하고 유니폼을 벗게 되면 평생 자신은 물론 롯데 팬들의 마음 한켠에 빈 곳이 남아 있게 될 것이다. 그는 2년 재계약을 하면서 은퇴 배수진을 치고 ‘우승을 위해 할 수 있는 모든 것을 다하겠다. 후배들에게 내가 가진 모든 것을 전수해주겠다’고 밝혔다. 그런데 프랜차이즈 스타 손아섭이 NC 다이노스로 떠나고 강민호 재영입도 실패하면서 롯데의 한국시리즈 우승은 점점 멀어져 가고 있다. 이대호의 꿈이 무너져 간다. 이제는 그의 침묵이 어색하다. [사진=마이데일리 DB](장윤호 기자 changyh218@mydaily.co.kr)"

In [242]:
import re

pattern = r"^\[.+?\]" 
pattern_second = r"^\(.+?\)" 

In [243]:
re.findall(pattern, example_article)

['[마이데일리 = 장윤호 기자]']

In [213]:
nums = [1, 2, 3, 4, 5, 6 ,7, 8, 9, 10]

In [296]:
random.choices(nums, k=2)

[10, 2]

### 전체 데이터 셋에서 일부를 샘플링해서 정규표현식 패턴에 일치하는 텍스트만 출력

In [307]:
import random

# random.seed(1234)

def check_pattern_match(data, pattern, num_samples=300):
    samples = random.choices(data, k=num_samples)
    for sample in samples:
        _, _, _, content = sample
        matches = re.findall(pattern, content)
        if matches:
            print(matches)

In [308]:
check_pattern_match(data, pattern)

['[OSEN=홍지수 기자]']
['[고척=스포츠조선 김영록 기자]']
['[수원=스포츠조선 박상경 기자]']
['[스포티비뉴스=잠실, 김민경 기자]']
['[OSEN=조형래 기자]']
['[KBO리그]']
['[스포티비뉴스=대전, 최민우 기자]']
['[마이데일리 = 잠실 윤욱재 기자]']
['[일간스포츠 배중현]']
['[데일리안 = 김태훈 기자]']
['[서울=뉴시스]']
['[서울=뉴시스]']
['[잠실=스포츠조선 이종서 기자]']
['[인천=뉴시스]']
['[OSEN=광주, 이선호 기자]']
['[스포티비뉴스=잠실, 박정현 기자]']
['[인천=뉴시스]']
['[스포티비뉴스=김태우 기자]']
['[서울=뉴시스]']
['[스포티비뉴스=사직, 고봉준 기자]']
['[광주=스포츠조선 김영록 기자]']
['[서울=뉴시스]']
['[소공동=스포츠조선 이승준 기자]']
['[일간스포츠 이형석]']
['[마이데일리 = 고척돔 박승환 기자]']
['[서울=뉴시스]']
['[OSEN=이선호 기자]']
['[OSEN=이후광 기자]']
['[서울=뉴시스]']
['[인천=스포츠조선 나유리 기자]']
['[OSEN=대전, 이상학 기자]']
['[OSEN=대구, 손찬익 기자]']
['[OSEN=한용섭 기자]']
['[OSEN=길준영 기자]']
['[이데일리 스타in 이석무 기자]']
['[OSEN=이선호 기자]']
['[스포티비뉴스=수원, 김태우 기자]']
['[스포티비뉴스=고봉준 기자]']
['[마이데일리 = 김진성 기자]']
['[마이데일리 = 곽경훈 기자]']
['[마이데일리 = 인천 김진성 기자]']
['[스포티비뉴스=고척, 박정현 기자]']
['[OSEN=고척, 길준영 기자]']
['[스포티비뉴스=잠실, 신원철 기자]']
['[스포티비뉴스=인천, 고봉준 기자]']
['[스포티비뉴스=목동, 최민우 기자]']
['[OSEN=인천, 홍지수 기자]']
['[서울=뉴시스]']
['[광주=스포츠조선 박상경 기자]']
['[서울=뉴시스]']
['[OSEN=잠실, 홍지수 기자]']

In [301]:
example_str = "hello world 123 nice to meet you 12"
remove_pattern = r"\d+"

In [251]:
re.findall(remove_pattern, example_str)

['123', '12']

In [252]:
re.sub(remove_pattern, "", example_str)

'hello world  nice to meet you '

### re.sub 함수를 이용해서 불필요한 텍스트 제거
앞서서 검증한 패턴을 가지고 데이터 셋에서 불필요한 부분을 제거

In [245]:
def filter_prefix(data, pattern):
    filtered_data = []
    for url, datetime_str, title, content in tqdm(data):
        filtered_content = re.sub(pattern, "", content)
        if not filtered_content:
            continue
        filtered_content = filtered_content.strip()
        filtered_data.append((url, datetime_str, title, filtered_content))
    return filtered_data

In [246]:
filtered_data = filter_prefix(data, pattern)

100%|██████████████████████████████████████████████████████████████████| 102520/102520 [00:00<00:00, 112475.65it/s]


In [247]:
filtered_data[:3]

[('https://sports.news.naver.com/news?oid=477&aid=0000346920',
  '20220314',
  '"시프트를 좋아하지만, 금지해도 야구의 후퇴는 아니다"',
  '메이저리그가 수비 시프트를 금지한다. 수비 시프트를 사랑하는 한화 이글스 카를로스 수베로 감독은 아쉬워하면서도 잃는 게 있으면 얻는 것도 있다고 설명했다.한화 수베로 감독은 13일 대구삼성라이온즈파크에서 삼성 라이온즈와 시범경기에 앞서 메이저리그 수비 시프트 금지에 대한 본인 생각을 이야기헀다. 메이저리그는 오는 2023년부터 수비 시프트를 금지한다. 새로운 CBA(노사협정) 개정으로 규칙이 바뀐 셈이다.KBO 리그에 변화는 없다. 그러나 메이저리그를 경험해본 수베로 감독은 시프트가 사라지는 것이 아쉽다. 그는 KBO 리그에서 가장 많은 시프트를 사용하는 감독 가운데 한 명이다. 타자 유형에 따라 공격적인 시프트를 보여준다. 2루수가 유격수 쪽에 치우치거나 우익수 쪽으로 나가는 것은 기본적인 시프트일 정도로 다양한 시프트를 활용해 타자의 안타 생산을 막는다.13일 수베로 감독은 "나는 수비 시프트를 좋아한다. 시프트는 야구의 일부다. 야구는 확률 싸움을 하는 스포츠다. 시프트는 확률에 따라 맞춰 움직여 아웃 카운트를 잡을 수 있다. 마음에 든다. 최근 생긴 방법도 아니다. 1980~1990년대부터 경기를 할 때 투수와 타자에 맞춰 수비 위치를 바꿨다. 투수의 투구와 타자의 타격 성향을 고려해 움직였다. 나는 시프트 자체를 높게 평가한다"며 야구를 잘하는 수단으로 시프트가 좋다고 목소리를 높였다.메이저리그에는 수베로 감독만큼 시프트를 즐겨 사용하는 감독들이 많다. 극단적으로 당겨치는 왼손 타자를 사대하기 위해 내야수 4명 전원을 1루와 2루 사이에 배치한 장면도 있다. 메이저리그 강타자 배리 본즈는 시프트가 있어도 신경 쓰지 않고 본인의 타격을 한 것으로 유명하다. 투수들은 간혹 정상 수비였다면 아웃되는 타구였는데, 시프트로 안타를 내주자 아쉬워하거나 화를 

### 날짜 순으로 정렬

In [317]:
from datetime import datetime, timedelta

example_date = datetime(2022, 10, 28)

In [311]:
example_date

datetime.datetime(2022, 10, 28, 0, 0)

In [315]:
example_date_two = datetime(2022, 7, 31)

In [316]:
example_date - example_date_two

datetime.timedelta(days=89)

In [318]:
example_date + timedelta(days=10)

datetime.datetime(2022, 11, 7, 0, 0)

In [322]:
example_date_str = "2022-10-28 12:23:41"

example_date_obj = datetime.strptime(example_date_str, "%Y-%m-%d %H:%M:%S")

In [325]:
example_date_obj.strftime("%m%Y")

'102022'

In [323]:
example_date_obj

datetime.datetime(2022, 10, 28, 12, 23, 41)

In [327]:
filtered_data[0]

('https://sports.news.naver.com/news?oid=477&aid=0000346920',
 '20220314',
 '"시프트를 좋아하지만, 금지해도 야구의 후퇴는 아니다"',
 '메이저리그가 수비 시프트를 금지한다. 수비 시프트를 사랑하는 한화 이글스 카를로스 수베로 감독은 아쉬워하면서도 잃는 게 있으면 얻는 것도 있다고 설명했다.한화 수베로 감독은 13일 대구삼성라이온즈파크에서 삼성 라이온즈와 시범경기에 앞서 메이저리그 수비 시프트 금지에 대한 본인 생각을 이야기헀다. 메이저리그는 오는 2023년부터 수비 시프트를 금지한다. 새로운 CBA(노사협정) 개정으로 규칙이 바뀐 셈이다.KBO 리그에 변화는 없다. 그러나 메이저리그를 경험해본 수베로 감독은 시프트가 사라지는 것이 아쉽다. 그는 KBO 리그에서 가장 많은 시프트를 사용하는 감독 가운데 한 명이다. 타자 유형에 따라 공격적인 시프트를 보여준다. 2루수가 유격수 쪽에 치우치거나 우익수 쪽으로 나가는 것은 기본적인 시프트일 정도로 다양한 시프트를 활용해 타자의 안타 생산을 막는다.13일 수베로 감독은 "나는 수비 시프트를 좋아한다. 시프트는 야구의 일부다. 야구는 확률 싸움을 하는 스포츠다. 시프트는 확률에 따라 맞춰 움직여 아웃 카운트를 잡을 수 있다. 마음에 든다. 최근 생긴 방법도 아니다. 1980~1990년대부터 경기를 할 때 투수와 타자에 맞춰 수비 위치를 바꿨다. 투수의 투구와 타자의 타격 성향을 고려해 움직였다. 나는 시프트 자체를 높게 평가한다"며 야구를 잘하는 수단으로 시프트가 좋다고 목소리를 높였다.메이저리그에는 수베로 감독만큼 시프트를 즐겨 사용하는 감독들이 많다. 극단적으로 당겨치는 왼손 타자를 사대하기 위해 내야수 4명 전원을 1루와 2루 사이에 배치한 장면도 있다. 메이저리그 강타자 배리 본즈는 시프트가 있어도 신경 쓰지 않고 본인의 타격을 한 것으로 유명하다. 투수들은 간혹 정상 수비였다면 아웃되는 타구였는데, 시프트로 안타를 내주자 아쉬워하거나 화를 내기도 

In [328]:
from datetime import datetime

sorted_data = sorted(filtered_data, key=lambda x: datetime.strptime(x[1], "%Y%m%d"))

### 결과 출력

In [333]:
with open("./data/baseball_preprocessed.csv", "w") as fw:
    writer = csv.writer(fw)
    writer.writerow(["url", "datetime_str", "title", "content"])
    for row in tqdm(sorted_data):
        writer.writerow(row)

100%|███████████████████████████████████████████████████████████████████| 102482/102482 [00:02<00:00, 47742.84it/s]
